# Tutorials on Entity Embedding
In this short tutorial, we demonstrate how to contruct entity embedding using vincinity information

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
EPS = 0.00001

## Note
This is the final implementation, but not the process.

In [19]:
class EntityEmbeddingLayer(nn.Module):
    def __init__(self, num_level, emdedding_dim, centroid):
        super(EntityEmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(num_level, embedding_dim)
        self.centroid = torch.tensor(centroid).detach_().unsqueeze(1)
    def forward(self, x): 
        """
        x must be batch_size times 1
        """
        x = x.unsqueeze(1)
        d = 1.0/((x-self.centroid).abs()+EPS)
        w = F.softmax(d.squeeze(2), 1)
        v = torch.mm(w, self.embedding.weight)
        return v
        

## Experimentation
This is the part where you experiment with the data

In [3]:
num_level = 10 # When configure dimension, do not use the same dimension for testing
embedding_dim = 5 
embedding = nn.Embedding(num_level, embedding_dim)

In [4]:
batch_size = 4
x = torch.randn(batch_size, 1)
centroid = torch.randn(num_level, 1)

In [5]:
embedding.weight.shape

torch.Size([10, 5])

In [10]:
x.shape

torch.Size([4, 1, 1])

In [11]:
centroid.shape

torch.Size([1, 10, 1])

In [9]:
x = x.unsqueeze(1)
centroid = centroid.unsqueeze(0) # This is not needed

In [12]:
(x- centroid).shape

torch.Size([4, 10, 1])

In [13]:
d = 1.0/((x-centroid).abs()+EPS)

In [14]:
w = F.softmax(d.squeeze(2), 1)

In [15]:
w.shape

torch.Size([4, 10])

In [16]:
w.sum()

tensor(4.)

In [17]:
v = torch.mm(w, embedding.weight)

In [18]:
v.shape

torch.Size([4, 5])

## Testing

In [20]:
x = torch.randn(batch_size, 1)
centroid = torch.randn(num_level) # This differs from the above definition
entity_embedding = EntityEmbeddingLayer(num_level, embedding_dim, centroid)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [21]:
entity_embedding(x)

tensor([[ 0.2952, -0.1353, -0.3040,  0.1783,  0.0248],
        [-0.6496, -2.2369, -1.2419,  1.3052, -1.2015],
        [ 1.0792,  1.3434,  0.5521,  0.6170,  0.6364],
        [ 0.2842, -0.1270, -0.2875,  0.1578,  0.0382]], grad_fn=<MmBackward>)